In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

sys.path.append(str(Path().cwd().parent.resolve()))

# Uncomment to get more debugging printouts:
"""
import logging

root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

"\nimport logging\n\nroot = logging.getLogger()\nroot.setLevel(logging.DEBUG)\n\nhandler = logging.StreamHandler(sys.stdout)\nhandler.setLevel(logging.DEBUG)\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nhandler.setFormatter(formatter)\nroot.addHandler(handler)\n"

In [2]:
from trulens_eval.keys import *

KEY SET: OPENAI_API_KEY
KEY SET: PINECONE_API_KEY
KEY SET: PINECONE_ENV
KEY SET: HUGGINGFACE_API_KEY
KEY SET: SLACK_TOKEN
KEY SET: SLACK_SIGNING_SECRET
KEY SET: COHERE_API_KEY


In [3]:
# LLama Index starter example from: https://gpt-index.readthedocs.io/en/latest/getting_started/starter_example.html

from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('llama_index/data').load_data()
index = GPTVectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()
# response = query_engine.query("What did the author do growing up?")
# print(response)

In [4]:
from trulens_eval import Tru, Query, Feedback
from trulens_eval import feedback

tru = Tru()
Tru().reset_database()

proc = Tru().start_dashboard(force=True, _dev=Path.cwd().parent)
thread = Tru().start_evaluator(restart=True)

Force stopping dashboard ...
killing psutil.Process(pid=68581, name='python3.8', status='running', started='20:27:29')
Starting dashboard ...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.1.60:8501 .
Looking for things to do. Stop me with `tru.stop_evaluator()`.

In [5]:
# For aggregation,
import numpy as np

# Construct feedback functions.

hugs = feedback.Huggingface()
openai = feedback.OpenAI()

# Language match between question/answer.
f_lang_match = Feedback(hugs.language_match).on(
    text1=Query.RecordInput, text2=Query.RecordOutput
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = Feedback(openai.relevance).on(
    prompt=Query.RecordInput, response=Query.RecordOutput
)

# Question/statement relevance between question and each context chunk.
f_qs_relevance = feedback.Feedback(openai.qs_relevance).on(
    question=Query.RecordInput,
    statement=Query.Record.model.retriever.retrieve.rets[:].node.text
).aggregate(np.min)

feedbacks = [f_lang_match, f_qa_relevance, f_qs_relevance]

l = tru.Llama(engine=query_engine, feedbacks=feedbacks)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface api: 0requests [00:00, ?requests/s]

openai api: 0requests [00:00, ?requests/s]

✅ app app_hash_9547295154a032e9657f963f6e0808e4 -> default.sqlite
✅ feedback def. feedback_definition_hash_26ce7668806e32a7eaa5b68afb93fe64 -> default.sqlite
✅ feedback def. feedback_definition_hash_f1e4e86a9acd060a3f1e19c33bd3ddb3 -> default.sqlite
✅ feedback def. feedback_definition_hash_5a404e27b46ff5495995c02ceb079c02 -> default.sqlite


In [6]:
list(l.instrumented())

[(JSONPath(), ['engine']),
 (JSONPath()._retriever, ['retriever']),
 (JSONPath().retriever, ['retriever'])]

In [7]:
res, record = l.query_with_record("What did the author do growing up?")

Token indices sequence length is longer than the specified maximum sequence length for this model (1808 > 1024). Running this sequence through the model will result in indexing errors


✅ record record_hash_b7b6d7914afc11e53725d39139a1b238 from app_hash_9547295154a032e9657f963f6e0808e4 -> default.sqlite


In [8]:
record.dict()

{'record_id': 'record_hash_b7b6d7914afc11e53725d39139a1b238',
 'app_id': 'app_hash_9547295154a032e9657f963f6e0808e4',
 'cost': {'n_tokens': 0, 'cost': 0.0},
 'perf': {'start_time': datetime.datetime(2023, 6, 14, 20, 27, 53, 931489),
  'end_time': datetime.datetime(2023, 6, 14, 20, 28, 0, 569940)},
 'ts': datetime.datetime(2023, 6, 14, 20, 28, 0, 569963),
 'tags': '',
 'main_input': 'What did the author do growing up?',
 'main_output': '\nThe author grew up writing essays, learning Italian, exploring Florence, painting people, working with computers, attending RISD, living in a rent-controlled apartment, building an online store builder, editing code, launching software, publishing essays online, writing essays, painting still life, working on spam filters, cooking for groups, and buying a building in Cambridge.',
 'main_error': 'None',
 'calls': [{'stack': ({'path': {'path': ({'item_or_attribute': 'model'},)},
     'method': {'obj': {'cls': {'name': 'RetrieverQueryEngine',
        'mod

Waiting for {'error': 'Model papluca/xlm-roberta-base-language-detection is currently loading', 'estimated_time': 44.49275207519531} (44.49275207519531) second(s).


Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.✅ feedback feedback_result_hash_d99138274135844a730d1d9b75123fce on record_hash_b7b6d7914afc11e53725d39139a1b238 -> default.sqlite
✅ feedback feedback_result_hash_8a4c2cdd057d6c6d8c54fbd7782479f3 on record_hash_b7b6d7914afc11e53725d39139a1b238 -> default.sqlite
✅ feedback feedback_result_hash_967f8dc3d903191189ec3f69f148a1a8 on record_hash_b7b6d7914afc11e53725d39139a1b238 -> default.sqlite
Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_evaluator()`.Looking for things to do. Stop me with `tru.stop_